In [58]:
#default_exp flowchatbot

In [1]:
# export
import redis
import enum
import json
import re

## Data Utils

In [44]:
# export
def get_chained_data(data, *keys)
    '''
    Gets data from dict hierarchy with given keys 
    (none if any link is missing)
    '''
    d = data
    for key in keys:
        if key not in d:
            return None
        d = d[key]
    return d

def set_chained_data(data, *keys, val=0):
    '''
    Sets data to dict hierarchy and creating links where necessary
    '''
    d = data
    for key in keys[:-1]:
        if key not in d:
            d[key] = {}
        d = d[key]
    d[keys[-1]] = val


In [46]:
# export
def tryint(x):
    '''
    Check if input can be parsed as int, return -1 otherwise
    '''
    try:
        return int(x)
    except:
        return -1
    

## Chatbot classes

In [4]:
# export
class CHAT_RET(enum.Enum):
    '''
    Movement values from chat segment.
    If return stay, bot should reask this question
    If next, bot should move to next in link
    '''
    STAY = 0
    NEXT = 1

In [5]:
# export
class Pipe:
    '''
    Base class for all flow element classes
    Sets up key, None as next link and callbacks for question and answer call
    '''
    def __init__(self, on_question=None, on_answer=None):
        self.key = ''
        self.next = None
        self.on_question = on_question
        self.on_answer = on_answer
    
    def __repr__(self):
        return f'{type(self)}: {self.key}'
    
    def question(self):
        pass
    
    def answer(self, resp):
        pass

In [6]:
# export
class Segment(Pipe):
    '''
    Basic segment unit, asks one question and gives one answer
    Stores response as it is in 'data' field
    '''
    def __init__(self, key, q, a, **kwargs):
        super().__init__(**kwargs)
        self.q = q
        self.a = a
        self.key = key        
        
    def question(self, data):
        if self.on_question:
            self.on_question(data)
        return {'txt': self.q}
    
    def answer(self, resp, data):
        data['data'] = resp
        if self.on_answer:
            self.on_answer(data)
        return CHAT_RET.NEXT, {'txt': self.a}
    

In [7]:
# export
class ValidatedSegment(Segment):
    '''
    Segment element with data validation facility
    Constructor takes validation function as arg.
    '''
    def __init__(self, key, q, a, errmsg, valid_fn):
        '''
        valid_fn is called with user response. If it returns true, data 
        is stored and bot moves forward, otherwise bot stays and shows error message
        '''
        super().__init__(key, q, a)
        self.err = errmsg
        self.valid_fn = valid_fn
         
    def answer(self, resp, data):
        if self.valid_fn(resp):
            return super().answer(resp, data)
        data['data'] = ''
        return CHAT_RET.STAY, {'txt': self.err}
  

In [ ]:
# export
class MultiChoiceSegment(Segment):
    '''
    Element for asking question with multi choice answers.
    Both response number and corresponding answer is stored in data
    '''
    def __init__(self, key, q, resp_lst, ans, **kwargs):
        '''
        resp_lst is list of choices to be given. Numbered from 1 to n
        '''
        super().__init__(key, q, ans, **kwargs)
        self.resp_lst = resp_lst
        
    def question(self, data):
        return {'txt': self.q + '\n' + '\n'.join([f'{i+1}. {q}' for (i,q) in enumerate(self.resp_lst)])}
    
    def answer(self, resp, data):
        if 0 < tryint(resp) <= len(self.resp_lst):
            data['data'] = (tryint(resp) - 1, self.resp_lst[tryint(resp) - 1])
            return CHAT_RET.NEXT, {'txt': self.a}
        return CHAT_RET.STAY, {'txt': f'Please enter one of 1..{len(self.resp_lst)} as answer'}
    

In [9]:
# export
class ComputeSegment(Segment):
    '''
    Segment to do computation. Data of parent elememnt is passed to answer function
    Should be inherited with overriden answer function to do actual computations
    '''
    def answer(self, resp, data):
        '''
        data of parent is passed
        '''
        return super().answer(resp, data)    

In [10]:
# export
class Composite(Pipe):
    '''
    Composite element, can have any other element including composite as children.
    All children form linked list and traversed sequentially while this element 
    will return STAY
    When end of linked list is reached, element returns NEXT
    '''
    def __init__(self, key, *args):
        '''
        args correspond to all children elements
        '''
        super().__init__()
        for i in range(len(args) - 1):
            args[i].next = args[i+1]
            
        self.args = args
        self.key = key
        
        self.nodes = {}
        for a in args:
            self.nodes[a.key] = a
    
    def getpos(self, data):
        pos = get_chained_data(data, 'pos')
        if pos is None:
            pos = self.args[0].key
            set_chained_data(data, 'pos', val=pos)
            
        return self.nodes[pos]
        
    def setpos(self, data, pos):
        set_chained_data(data, 'pos', val=pos.key)
        
    def getdata(self, data, key):
        val = get_chained_data(data, key)
        if val is None:
            val = {}
            set_chained_data(data, key, val=val)
        return val
        
    def question(self, data):
        cur = self.getpos(data)
        print(f'composite question current {cur.key} data {data}')
        curdata = self.getdata(data, cur.key)
            
        if isinstance(cur, Splitter):
            q = cur.question(data)
        else:
            q = cur.question(curdata)
            
        return q
    
    def move(self, cur, cmd, data):
        if cmd == CHAT_RET.NEXT:
            nxt = cur.next
            if nxt == None: # self.args[-1]:
                data.pop('pos')
                return CHAT_RET.NEXT
        else:
            nxt = cur
            
        self.setpos(data, nxt)
        return CHAT_RET.STAY
    
    def answer(self, resp, data):
        cur = self.getpos(data)
        print(f'composite answer current {cur.key} data {data}')
        curdata = self.getdata(data, cur.key)
       
        if isinstance(cur, SkipSplitter):
            if cur.decider_fn(data) == 1:
                mov, ans = cur.answer(resp, data)
                return self.move(cur, mov, data), ans    
            else:
                cur = cur.next
                self.setpos(data, cur)
                curdata = self.getdata(data, cur.key)
                
        if isinstance(cur, Splitter):
            mov, ans = cur.answer(resp, data)
        elif isinstance(cur, ComputeSegment):
            mov, ans = cur.answer(resp, data)
        else:
            mov, ans = cur.answer(resp, curdata)        
        return self.move(cur, mov, data), ans    


In [ ]:
#export
class LoopComposite(Composite):
    '''
    Composite element with ability to start from begin once end is reached.
    Should have a Skip element before to break loop
    '''
    def move(self, cur, cmd, data):
        if cmd == CHAT_RET.NEXT:
            nxt = cur.next
            if nxt == None: # self.args[-1]:
                data.pop('pos')
                return CHAT_RET.STAY
        else:
            nxt = cur
            
        self.setpos(data, nxt)
        return CHAT_RET.STAY

In [28]:
# export
class TextAdapter(Pipe):
    '''
    Text based adapter to invoke root element on user reponse.
    A similar adapter is needed for HTML or other bots
    '''
    def __init__(self, root):
        super().__init__()
        self.root = root
        self.r = redis.Redis()
    
    def respond(self, inp, session):
        if self.r.exists(session):
            data = json.loads(self.r.get(session))
        else:
            data = {}
        n, d1 = self.root.answer(inp, data)
#         if n == CHAT_RET.NEXT:
#             data.pop('pos')
        d2 = self.root.question(data)
        print(f'adapter data {data}')
        self.r.set(session, json.dumps(data))
        return f"{d1['txt']}\n{d2['txt']}"

## Test segment and composite

In [29]:
bot = TextAdapter(
    Composite(
        'c1',
        Segment('s0', '', 'welcome'),
         Composite('c2', 
            Segment('s1', 'your name', 'got it'),
            ValidatedSegment('s2', 'your phone number', 'got it', 
                            'phone number should be 10 digits with optional +91 at beginning',
                            lambda x: re.match('(\+91)?\d{10}$', x) is not None),        
            Segment('s3', 'your email', 'ok')
                  ),
        Composite('c3', 
            Segment('s4', 'your income', 'ok')
                 )
        
    )
)
bot.r.flushall()

True

In [ ]:
bot.respond('hi', 3)

In [ ]:
bot.respond('Niraj', 3)

In [ ]:
bot.respond('123123', 3)

In [ ]:
bot.respond('1231234567', 3)

In [ ]:
bot.respond('asd@qwe', 3)

In [ ]:
bot.respond('30000', 3)

In [ ]:
bot.respond('hi', 3)

In [ ]:
bot.respond('Niraj', 3)

In [ ]:
bot.respond('123123', 3)

In [ ]:
bot.respond('1231234567', 3)

In [ ]:
bot.respond('asd@qwe', 3)

In [ ]:
bot.respond('30000', 3)

## Splitters

In [30]:
# export
class Splitter(Pipe):
    '''
    Splitter element to branch flow based on some condition. 
    '''
    def __init__(self, key, *branches, decider_fn=lambda _: 0, default = 0):
        '''
        branches are all possible branch elements (could be composites)
        decider_fn is passes data of parent and should return branch index 
        (between 0 to n-1)
        default index is used if invalid index is returned by decider_fn
        '''
        super().__init__()
        self.key = key
        self.branches = branches
        self.nodes = {}
        for a in branches:
            self.nodes[a.key] = a
            
        self.default = default
        self.decider_fn = decider_fn
        
    def decide(self, data):
        '''
        Based on parent level data decide branch and return branch key
        '''
        val = self.decider_fn(data)
        print(f'decider {self.key} {val}')
        if val >= len(self.branches):
            val = self.default
        self.branches[val].next = self.next
        return self.branches[val].key
    
    def question(self, data):
        '''
        Parent data for splitter
        '''
        print(f'splitter question data {data}')
        pos = self.decide(data)
        set_chained_data(data, self.key, 'pos', val = pos)
        print(f'splitter question data after pos set {self.key} {pos} {data}')
        cur = self.nodes[pos]
        curdata = get_chained_data(data, self.key, cur.key)
        if curdata is None:
            curdata = {}
            set_chained_data(data, self.key, cur.key, val=curdata)
        return cur.question(curdata)
    
    def answer(self, resp, data):
        print(f'splitter answer data {data}')
        pos = self.decide(data)
        set_chained_data(data, self.key, 'pos', val = pos)
    
        cur = self.nodes[pos]
        
        curdata = get_chained_data(data, self.key, cur.key)
        if curdata is None:
            curdata = {}
            set_chained_data(data, self.key, cur.key, val=curdata)
        
        mov, ans = cur.answer(resp, curdata)
        
        if mov == CHAT_RET.NEXT:
            return CHAT_RET.NEXT, ans
            
        return CHAT_RET.STAY, ans    


In [31]:
# export
class SkipSplitter(Splitter):
    '''
    A branch is taken on condition otherwise skips the branch 
    (name taken from skipconnection in resnets)
    '''
    def __init__(self, key, branch, decider_fn=lambda _: 0, default = 0):
        '''
        Single branch should be passed
        if decide_fn returns 1, branch will be taken otherwise skipped
        '''
        super().__init__(key, branch, decider_fn=decider_fn, default=default)
    
    def decide(self, data):
        '''
        Based on parent level data decide if branch to be taken
        '''
        val = self.decider_fn(data)
        return val
    
    def getpos(self, data):
        if (self.key in data) and  ('pos' in data[self.key]):
            return self.nodes[data[self.key]['pos']]
        
        data[self.key]['pos'] = self.branches[0].key
        return self.branches[0]
    
    def setpos(self, data, pos):
        data['pos'] = pos.key

    def getdata(self, data, key):
        if self.key in data:
            if key in data[self.key]:
                return data[self.key][key]
            else:
                data[self.key][key] = {}
                return data[self.key][key]
        data[self.key] = {}
        data[self.key][key] = {}
        return data[self.key][key]
    
    def question(self, data):
        '''
        Parent data for splitter
        '''
        print(f'skip question data {data}')
        if self.decide(data) == 1:
            # take branch
            cur = self.getpos(data)
            curdata = self.getdata(data, cur.key)
            return cur.question(curdata)
        else:
            # skip branch
            data['pos'] = self.next.key
            return self.next.question({})
     
    def answer(self, resp, data):
        print(f'skip answer data {data}')
        cur = self.getpos(data)
        curdata = self.getdata(data, cur.key)
            
        mov, ans = cur.answer(resp, curdata)
        
        if mov == CHAT_RET.NEXT:
            return CHAT_RET.NEXT, ans
            
        return CHAT_RET.STAY, ans    


## Test splitters

In [ ]:
bot.r.keys('*')

In [32]:
def test_fn(data):
    print('splitter decider', data, data['s4']['data'])
    if tryint(data['s4']['data']) < 10000:
        return 0
    return 1

In [33]:
def infodecider(data):
#     print(f'decider {data}')
    val = get_chained_data(data, 'ss1', 'c2', 's3', 'data')
    if (val is not None) and (val != ''):
        return 0
    return 1

In [47]:
bot = TextAdapter(
    Composite(
        'c1',
        Segment('s0', '', 'welcome'),
        SkipSplitter('ss1', 
                    Composite('c2', 
                        Segment('s1', 'your name', 'got it'),
                        ValidatedSegment('s2', 'your mobile', 'got it', '10 digit mobile',
                                    valid_fn=lambda x: re.match('(\+91)?\d{10}$', x) is not None),
                        ValidatedSegment('s3', 'your email', 'got it', 'email in format abc@qwe.zxc',
                                    valid_fn=lambda x: re.match('[a-zA-Z0-9._]+@[a-zA-Z0-9_]+\.[a-zA-Z0-9._]+', 
                                                                x) is not None)
                    ),
                    decider_fn= infodecider),
        Segment('s4', 'your income', 'ok'),
        Splitter('s5', 
                Segment('s51', 'Do you have bike?', 'ok'),
                Segment('s52', 'Do you have car?', 'ok'),
                decider_fn=test_fn)
    )
)
bot.r.flushall()

True

In [56]:
bot.respond('hi', 3)

composite answer current s0 data {'s0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '11000'}, 's5': {'pos': 's52', 's52': {'data': 'No'}}, 'pos': 's0'}
composite question current ss1 data {'s0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '11000'}, 's5': {'pos': 's52', 's52': {'data': 'No'}}, 'pos': 'ss1'}
skip question data {'s0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '11000'}, 's5': {'pos': 's52', 's52': {'data': 'No'}}, 'pos': 'ss1'}
adapter data {'s0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '11000'}, 's5': {'pos': 's52', 's52': {'data': 'No'}}, 'pos': 's4'}


'welcome\nyour income'

In [49]:
bot.respond('Niraj', 3)

composite answer current ss1 data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's1', 's1': {}}}}
skip answer data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's1', 's1': {}}}}
composite answer current s1 data {'pos': 's1', 's1': {}}
composite question current ss1 data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}}}}
skip question data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}}}}
composite question current s2 data {'pos': 's2', 's1': {'data': 'Niraj'}}
adapter data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {}}}}


'got it\nyour mobile'

In [50]:
bot.respond('123123', 3)

composite answer current ss1 data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {}}}}
skip answer data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {}}}}
composite answer current s2 data {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {}}
composite question current ss1 data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {'data': ''}}}}
skip question data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {'data': ''}}}}
composite question current s2 data {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {'data': ''}}
adapter data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {'data': ''}}}}


'10 digit mobile\nyour mobile'

In [51]:
bot.respond('1231234567', 3)

composite answer current ss1 data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {'data': ''}}}}
skip answer data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {'data': ''}}}}
composite answer current s2 data {'pos': 's2', 's1': {'data': 'Niraj'}, 's2': {'data': ''}}
composite question current ss1 data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}}}}
skip question data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}}}}
composite question current s3 data {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}}
adapter data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {}}}}


'got it\nyour email'

In [52]:
bot.respond('asd@qwe', 3)

composite answer current ss1 data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {}}}}
skip answer data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {}}}}
composite answer current s3 data {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {}}
composite question current ss1 data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': ''}}}}
skip question data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': ''}}}}
composite question current s3 data {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': ''}}
adapter data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {

'email in format abc@qwe.zxc\nyour email'

In [53]:
bot.respond('asd@qwe.com', 3)

composite answer current ss1 data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': ''}}}}
skip answer data {'pos': 'ss1', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': ''}}}}
composite answer current s3 data {'pos': 's3', 's1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': ''}}
composite question current s4 data {'pos': 's4', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}}
adapter data {'pos': 's4', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {}}


'got it\nyour income'

In [57]:
bot.respond('9000', 3)

composite answer current s4 data {'s0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '11000'}, 's5': {'pos': 's52', 's52': {'data': 'No'}}, 'pos': 's4'}
composite question current s5 data {'s0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '9000'}, 's5': {'pos': 's52', 's52': {'data': 'No'}}, 'pos': 's5'}
splitter question data {'s0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '9000'}, 's5': {'pos': 's52', 's52': {'data': 'No'}}, 'pos': 's5'}
splitter decider {'s0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '9000'}, 's5': {'pos': 's52', 's52': {'data': 'No'}}, 'pos': 's5'} 9000
de

'ok\nDo you have bike?'

In [55]:
bot.respond('No', 3)

composite answer current s5 data {'pos': 's5', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '11000'}, 's5': {'pos': 's52', 's52': {}}}
splitter answer data {'pos': 's5', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '11000'}, 's5': {'pos': 's52', 's52': {}}}
splitter decider {'pos': 's5', 's0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '11000'}, 's5': {'pos': 's52', 's52': {}}} 11000
decider s5 1
composite question current s0 data {'s0': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s1': {'data': 'Niraj'}, 's2': {'data': '1231234567'}, 's3': {'data': 'asd@qwe.com'}}}, 's4': {'data': '11000'}, 's5': {'pos': 's52', 's52': {'data': 'No'}}, 'pos': 's0'}
adapter data {'s0': {'d

'ok\n'

In [ ]:
bot.respond('ok', 3)

In [ ]:
bot.r.get(3)

In [ ]:
r = redis.Redis()
r.exists(session_id)
r.set(session_id, '{}')
r.expire(session_id, 300)
r.get(session_id)

In [ ]:
re.match('(\+91)?\d{10}$', '+911231231231') is not None

In [ ]:
!pip install redis

In [ ]:
y

## Build Lib

In [3]:
from nbdev.export import notebook2script 
notebook2script()

Converted multiprocess-bot.ipynb.
Converted oil-crm.ipynb.
Converted scratchpad.ipynb.
